In [28]:
from bertopic import BERTopic
import tweepy
import pandas as pd
import numpy as np
import csv
import re
pd.set_option("display.max_colwidth", None)
from collections import Counter
import scipy.sparse as sp
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import itertools
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
#path=r'C:\Users\ddl11\Desktop\project\tweets'   #load data
dem1 = pd.read_csv('dems_tweets.csv')
dem1 = dem1[['user_id','Time','Text']]
dem2 = pd.read_csv('dems_tweets_2.csv')
dem2 = dem2[['user_id','Time','Text']]
dems = pd.concat([dem1,dem2])
dems = dems.reset_index()
dems = dems[['user_id','Time','Text']]

#rep1 = pd.read_csv('reps_tweets.csv')
#rep1 = rep1[['user_id','Time','Text']]
#rep2 = pd.read_csv('reps_tweets_2.csv')
#rep2 = rep2[['user_id','Time','Text']]
#reps = pd.concat([rep1,rep2])
#reps = reps.reset_index()
#reps = reps[['user_id','Time','Text']]

In [30]:
dems['Time'] = pd.to_datetime(dems['Time'])
dems = dems[(dems['Time'] > '2020-10-31') & (dems['Time'] < '2021-07-27')]
#reps['Time'] = pd.to_datetime(reps['Time'])
#reps = reps[(reps['Time'] > '2020-10-31') & (reps['Time'] < '2021-07-27')]

In [31]:
# first preprecessing
dems['Text'] = dems['Text'].apply(lambda x:x[2:])  #edit raw text
#reps['Text'] = reps['Text'].apply(lambda x:x[2:])
dems['Text'] = dems['Text'].apply(lambda x:x[:-1])
#reps['Text'] = reps['Text'].apply(lambda x:x[:-1])

dems['Text'] = dems['Text'].str.replace(r'\\xe2\\x80\\x99', '\'') #replace some unicodes
dems['Text'] = dems['Text'].str.replace(r'\\xe2\\x80\\x9c', '\"')
dems['Text'] = dems['Text'].str.replace(r'\\xe2\\x80\\x9d', '\"')
#reps['Text'] = reps['Text'].str.replace(r'\\xe2\\x80\\x99', '\'')
#reps['Text'] = reps['Text'].str.replace(r'\\xe2\\x80\\x9c', '\"')
#reps['Text'] = reps['Text'].str.replace(r'\\xe2\\x80\\x9d', '\"')

dems['Text'] = dems['Text'].str.replace(r'\\x[A-Za-z0-9./]+', '')
#reps['Text'] = reps['Text'].str.replace(r'\\x[A-Za-z0-9./]+', '')#remove all other unicodes

dems['Text'] = dems['Text'].str.replace(r'\\n', ' ')
#reps['Text'] = reps['Text'].str.replace(r'\\n', ' ')

In [32]:
#regular expressions
dems['Text'] = dems.Text.str.lower()
dems['Text'] = dems.Text.apply(lambda x: re.sub(r'\b(lol)\b', 'happyface', x))
dems['Text'] = dems.Text.apply(lambda x: re.sub(r'https?:\/\/\S+', ' ', x))
dems['Text'] = dems.Text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", ' ', x))
dems['Text']= dems.Text.apply(lambda x: re.sub("@[A-Za-z0-9_]+", ' ', x))
dems['Text']= dems.Text.apply(lambda x: re.sub("#[A-Za-z0-9_]+", ' ', x))
dems['Text'] = dems.Text.apply(lambda x: re.sub(r"\b[0-9]+\b", ' ', x))
dems['Text'] = dems.Text.apply(lambda x: re.sub(r"[^A-Za-z0-9 ]",'', x))
dems['Text']= dems.Text.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", ' ', x))
dems['Text'] = dems.Text.apply(lambda x: re.sub(r"\b[a-z0-9]\b",' ', x))
dems['Text'] = dems.Text.apply(lambda x: re.sub(' +', ' ', x))

#reps['Text'] = reps.Text.str.lower()
#reps['Text'] = reps.Text.apply(lambda x: re.sub(r'\b(lol)\b', 'happyface', x))
#reps['Text'] = reps.Text.apply(lambda x: re.sub(r'https?:\/\/\S+', ' ', x))
#reps['Text'] = reps.Text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", ' ', x))
#reps['Text']= reps.Text.apply(lambda x: re.sub("@[A-Za-z0-9_]+", ' ', x))
#reps['Text']= reps.Text.apply(lambda x: re.sub("#[A-Za-z0-9_]+", ' ', x))
#reps['Text'] = reps.Text.apply(lambda x: re.sub(r"\b[0-9]+\b", ' ', x))
#reps['Text'] = reps.Text.apply(lambda x: re.sub(r"[^A-Za-z0-9 ]",'', x))
#reps['Text']= reps.Text.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", ' ', x))
#reps['Text'] = reps.Text.apply(lambda x: re.sub(r"\b[a-z0-9]\b",' ', x))
#reps['Text'] = reps.Text.apply(lambda x: re.sub(' +', ' ', x))


In [33]:
#reps = reps[reps["Text"] != ' '];reps = reps[reps["Text"] != '']
dems = dems[dems["Text"] != ' '];dems = dems[dems["Text"] != '']
#reps.index = range(len(reps));
dems.index = range(len(dems))

In [7]:
dems = dems.groupby('Time').sample(frac=.7733) #keep dems reps the same size

In [8]:
dems = dems.sample(660000,replace=True)
#reps = reps.sample(660000,replace=True)
dems.index = range(len(dems))
#reps.index = range(len(reps))

In [9]:
docs = dems['Text']
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english", min_df=10)
model = BERTopic(vectorizer_model=vectorizer_model,nr_topics="auto",low_memory=True,top_n_words=20,
                 min_topic_size=300,verbose=True)
topics, probabilities = model.fit_transform(docs)

2021-08-07 09:37:43,206 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2021-08-07 10:33:08,238 - BERTopic - Reduced dimensionality with UMAP
2021-08-07 10:38:50,450 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2021-08-07 10:40:35,000 - BERTopic - Reduced number of topics from 227 to 135


In [25]:
model.get_topic(97)

[('china', 0.1660926476014176),
 ('chinese', 0.08281270387423889),
 ('hong', 0.045425947238850115),
 ('hong kong', 0.04211122871988331),
 ('kong', 0.04140854079393119),
 ('chinas', 0.030206990181393042),
 ('tariffs', 0.028949925565671105),
 ('korea', 0.024627890473090917),
 ('north korea', 0.02213193145417531),
 ('xi', 0.014078272764133185),
 ('taiwan', 0.011349444418536504),
 ('human rights', 0.011016273214463506),
 ('government', 0.01076760955393996),
 ('war', 0.00827780366914058),
 ('qatar', 0.0082094720975641),
 ('allies', 0.008192164607000353),
 ('doesnt pay', 0.008061763786086909),
 ('russia', 0.00783571512715824),
 ('genocide', 0.007713403216593489),
 ('camps', 0.007572924209583971)]

In [26]:
model.get_topics()

{-1: [('capitol', 0.004038748200167018),
  ('senate', 0.003438689804697903),
  ('congress', 0.0031452227311417552),
  ('election', 0.0030712277743350776),
  ('january', 0.0029701944762702973),
  ('republican', 0.0029615797559760662),
  ('trumps', 0.0029109909911041714),
  ('republicans', 0.0028994804408496227),
  ('democracy', 0.0028933374475570827),
  ('voting', 0.0028916460402258334),
  ('vote', 0.002863014275986884),
  ('insurrection', 0.0028191309186286085),
  ('president', 0.0028170672913868425),
  ('state', 0.002779079531195402),
  ('today', 0.002764203963770627),
  ('american', 0.002732831965030902),
  ('donald', 0.0026942663160011643),
  ('rights', 0.002658129369014579),
  ('gop', 0.002653951758385869),
  ('america', 0.0026271258068062387)],
 0: [('vaccinated', 0.03223062799585185),
  ('vaccine', 0.03213017156623841),
  ('covid', 0.02895386595566259),
  ('vaccines', 0.01992644666423032),
  ('mask', 0.017460656105029067),
  ('vaccination', 0.014546090255826963),
  ('delta', 0.01

In [27]:
model.get_topic_freq().values.tolist()

[[-1, 319710],
 [0, 28813],
 [1, 21584],
 [2, 16773],
 [3, 12735],
 [4, 12651],
 [5, 12086],
 [6, 11347],
 [7, 8098],
 [8, 7335],
 [9, 5662],
 [10, 5240],
 [11, 5211],
 [12, 5090],
 [13, 4946],
 [14, 4713],
 [15, 4633],
 [16, 4326],
 [17, 4103],
 [18, 4085],
 [19, 4036],
 [20, 4026],
 [21, 3874],
 [22, 3843],
 [23, 3661],
 [24, 3654],
 [25, 3547],
 [26, 3470],
 [27, 3420],
 [28, 3416],
 [29, 3407],
 [30, 3127],
 [31, 3114],
 [32, 3100],
 [33, 3019],
 [34, 2914],
 [35, 2901],
 [36, 2843],
 [37, 2633],
 [38, 2607],
 [39, 2591],
 [40, 2451],
 [41, 2297],
 [42, 2295],
 [43, 2216],
 [44, 2178],
 [45, 2132],
 [46, 2034],
 [47, 1959],
 [48, 1937],
 [49, 1933],
 [50, 1883],
 [51, 1834],
 [52, 1631],
 [53, 1620],
 [54, 1618],
 [55, 1584],
 [56, 1512],
 [57, 1425],
 [58, 1423],
 [59, 1412],
 [60, 1406],
 [61, 1396],
 [62, 1394],
 [63, 1375],
 [64, 1329],
 [65, 1291],
 [66, 1263],
 [67, 1259],
 [68, 1141],
 [69, 1136],
 [70, 1089],
 [71, 1077],
 [72, 1063],
 [73, 1033],
 [74, 1026],
 [75, 1019],


In [ ]:
#model.save('reps_topic_model')

In [35]:
dems['Topic'] = 0

In [39]:
#reload data, run load code above, now len reps = 1000000+
dems['Topic'][700000:] = dems['Text'][700000:].apply(lambda x: model.find_topics(x)[0][0])

In [41]:
def deal_str(data):
    data = str(data)+'\t'
    return data
dems['user_id'] = dems['user_id'].map(deal_str)

dems.to_csv('dems_with_topics.csv')